# **BERT для вопросно-ответных систем**

##### ^^Скачайте датасет (SQuAD) отсюда. Для выполенения семинара Вам понадобятся файлы train-v2.0.json и dev-v2.0.json.**

##### **Склонируйте репозиторий https://github.com/huggingface/transformers (воспользуйтесь скриптом clone_pytorch_transformers.sh) и положите путь до папки examples в переменную PATH_TO_EXAMPLES.**

In [ ]:
! git clone https://github.com/huggingface/transformers.git

In [ ]:
!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('./stepik-dl-nlp')

In [ ]:
PATH_TO_TRANSFORMERS_REPO = 'https://github.com/huggingface/transformers.git'

In [ ]:
!gdown https://drive.google.com/uc?id=1-DR30q7MF-gZ51TDx596dAOhgh-uOAPj

In [ ]:
import os
os.environ['PATH_TO_TRANSFORMER_REPO'] = PATH_TO_TRANSFORMERS_REPO

In [ ]:
!pip install pytorch-transformers

In [ ]:
!wget 'https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad.py'
!wget 'https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad_evaluate.py'
!wget 'https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/run_squad.py'

In [ ]:
!git clone $PATH_TO_TRANSFORMERS_REPO

In [ ]:
import sys

PATH_TO_EXAMPLES = os.path.join(PATH_TO_TRANSFORMERS_REPO, 'examples')
sys.path.append(PATH_TO_EXAMPLES)

In [ ]:
import torch
import tqdm
import json

from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)

from run_squad import train, load_and_cache_examples

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

from transformers import (WEIGHTS_NAME, BertConfig, XLNetConfig, XLMConfig,
                          BertForQuestionAnswering, BertTokenizer)

from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

In [ ]:
!wget https://drive.google.com/drive/folders/1-DR30q7MF-gZ51TDx596dAOhgh-uOAPj?usp=sharing

In [ ]:
if torch.cuda.is_available():
    model.cuda()
    model.load_state_dict(torch.load('../input/bert-squad-5-epochs/bert_squad_5epochs.pt')) # если у вас есть GPU
else:
    model.load_state_dict(torch.load('../input/bert-squad-5-epochs/bert_squad_5epochs.pt', map_location=device)) # если GPU нет

#### **Дообучение**

In [ ]:
!pip install dataclasses

In [ ]:
from dataclasses import dataclass

@dataclass
class TRAIN_OPTS:
    train_file : str = '../input/squad-20/train-v2.0.json'    # SQuAD json-файл для обучения
    predict_file : str = '../input/squad-20/dev-v2.0.json'    # SQuAD json-файл для тестирования
    model_type : str = 'bert'               # тип модели (может быть  'bert', 'xlnet', 'xlm', 'distilbert')
    model_name_or_path : str = 'bert-base-uncased' # путь до предобученной модели или название модели из ALL_MODELS
    output_dir : str = '/tmp' # путь до директории, где будут храниться чекпоинты и предсказания модели
    device : str = 'cuda' # cuda или cpu
    n_gpu : int = 1 # количество gpu для обучения
    cache_dir : str = '' # где хранить предобученные модели, загруженные с s3
        
    # Если true, то в датасет будут включены вопросы, на которые нет ответов.
    version_2_with_negative : bool = True
    # Если (null_score - best_non_null) больше, чем порог, предсказывать null.
    null_score_diff_threshold : float = 0.0
    # Максимальная длина входной последовательности после WordPiece токенизации. Sequences 
    # Последовательности длиннее будут укорочены, для более коротких последовательностей будет использован паддинг
    max_seq_length : int = 384
    # Сколько stride использовать при делении длинного документа на чанки
    doc_stride : int = 128
    # Максимальное количество токенов в вопросе. Более длинные вопросы будут укорочены до этой длины
    max_query_length : int = 128 #
        
    do_train : bool = True
    do_eval : bool = True
        
    # Запускать ли evaluation на каждом logging_step
    evaluate_during_training : bool = True
    # Должно быть True, если Вы используете uncased модели
    do_lower_case : bool = True #
    
    per_gpu_train_batch_size : int = 8 # размер батча для обучения
    per_gpu_eval_batch_size : int = 8 # размер батча для eval
    learning_rate : float = 5e-5 # learning rate
    gradient_accumulation_steps : int = 1 # количество шагов, которые нужно сделать перед backward/update pass
    weight_decay : float = 0.0 # weight decay
    adam_epsilon : float = 1e-8 # эпсилон для Adam
    max_grad_norm : float = 1.0 # максимальная норма градиента
    num_train_epochs : float = 5.0 # количество эпох на обучение
    max_steps : int = -1 # общее количество шагов на обучение (override num_train_epochs)
    warmup_steps : int = 0 # warmup 
    n_best_size : int = 5 # количество ответов, которые надо сгенерировать для записи в nbest_predictions.json
    max_answer_length : int = 30 # максимально возможная длина ответа
    verbose_logging : bool = True # печатать или нет warnings, относящиеся к обработке данных
    logging_steps : int = 5000 # логировать каждые X шагов
    save_steps : int = 5000 # сохранять чекпоинт каждые X шагов
        
    # Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number
    eval_all_checkpoints : bool = True
    no_cuda : bool = False # не использовать CUDA
    overwrite_output_dir : bool = True # переписывать ли содержимое директории с выходными файлами
    overwrite_cache : bool = True # переписывать ли закешированные данные для обучения и evaluation
    seed : int = 42 # random seed
    local_rank : int = -1 # local rank для распределенного обучения на GPU
    fp16 : bool = False # использовать ли 16-bit (mixed) precision (через NVIDIA apex) вместо 32-bit"
    # Apex AMP optimization level: ['O0', 'O1', 'O2', and 'O3'].
    # Подробнее тут: https://nvidia.github.io/apex/amp.html
    fp16_opt_level : str = '01'

In [ ]:
dir(BertConfig)

In [ ]:
ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) for conf in (BertConfig, XLNetConfig, XLMConfig)), ())
ALL_MODELS

In [ ]:
args = TRAIN_OPTS()
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)

In [ ]:
train(args, train_dataset, model, tokenizer)

#### **Сохраняем веса дообученной модели на диск, чтобы в следующий раз не обучать модель заново.**

In [ ]:
torch.save(model.state_dict(), './stepik-dl-nlp/models/bert_squad_final_5epoch.pt')

#### **Подгрузить веса модели можно так:**

In [ ]:
model.load_state_dict(torch.load('../input/bert-squad-5-epochs/bert_squad_5epochs.pt'))

## **Оценка качества работы модели**

In [ ]:
PATH_TO_DEV_SQUAD = '../input/squad-20/dev-v2.0.json'
PATH_TO_SMALL_DEV_SQUAD = '../input/squad-20/train-v2.0.json'

with open(PATH_TO_DEV_SQUAD, 'r') as iofile:
    full_sample = json.load(iofile)
    
small_sample = {
    'version': full_sample['version'],
    'data': full_sample['data'][:1]
}

with open(PATH_TO_SMALL_DEV_SQUAD, 'w') as iofile:
    json.dump(small_sample, iofile)

In [ ]:
max_seq_length = 384
outside_pos = max_seq_length + 10
doc_stride = 128
max_query_length = 64
max_answer_length = 30

In [ ]:
examples = read_squad_examples(
    input_file=PATH_TO_SMALL_DEV_SQUAD,
    is_training=False,
    version_2_with_negative=True)

features = convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=False,
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    cls_token_at_end=False
)

input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

example_index = torch.arange(input_ids.size(0), dtype=torch.long)
dataset = TensorDataset(input_ids, input_mask, segment_ids, example_index, cls_index, p_mask)

In [ ]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

In [ ]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

all_results = []
for idx, batch in enumerate(tqdm.tqdm_notebook(eval_dataloader, desc="Evaluating")):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }
        inputs['token_type_ids'] = batch[2]
        example_indices = batch[3]
        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        result = RawResult(unique_id    = unique_id,
                           start_logits = to_list(outputs[0][i]),
                           end_logits   = to_list(outputs[1][i]))
        all_results.append(result)

In [ ]:
all_results[0]

In [ ]:
n_best_size = 5
do_lower_case = True
output_prediction_file = 'output_1best_file'
output_nbest_file = 'output_nbest_file'
output_na_prob_file = 'output_na_prob_file'
verbose_logging = True
version_2_with_negative = True
null_score_diff_threshold = 0.0

In [ ]:
# Генерируем файл с n лучшими ответами `output_nbest_file`
write_predictions(examples, features, all_results, n_best_size,
                    max_answer_length, do_lower_case, output_prediction_file,
                    output_nbest_file, output_na_prob_file, verbose_logging,
                    version_2_with_negative, null_score_diff_threshold)

In [ ]:
# Считаем метрики используя официальный SQuAD script
evaluate_options = EVAL_OPTS(data_file=PATH_TO_SMALL_DEV_SQUAD,
                             pred_file=output_prediction_file,
                             na_prob_file=output_na_prob_file)
results = evaluate_on_squad(evaluate_options)

#### **Посмотрим глазами на вопросы и предсказанные БЕРТом ответы:**

In [ ]:
with open('output_nbest_file', 'r') as iofile:
    predicted_answers = json.load(iofile)

In [ ]:
questions = {}
for paragraph in small_sample['data'][0]['paragraphs']:
    for question in paragraph['qas']:
        questions[question['id']] = {
            'question': question['question'],
            'answers': question['answers'],
            'paragraph': paragraph['context']
        }

In [ ]:
for q_num, (key, data) in enumerate(predicted_answers.items()):
    gt = '' if len(questions[key]['answers']) == 0 else questions[key]['answers'][0]['text']
    print('Вопрос {0}:'.format(q_num+1), questions[key]['question'])
    print('-----------------------------------')
    print('Ground truth:', gt)
    print('-----------------------------------')   
    print('Ответы, предсказанные БЕРТом:')
    preds = ['{0}) '.format(ans_num + 1) + answer['text'] + \
             ' (уверенность {0:.2f}%)'.format(answer['probability']*100) \
             for ans_num, answer in enumerate(data)]
    print('\n'.join(preds))
#     print('-----------------------------------')   
#     print('Параграф:', questions[key]['paragraph'])
    print('\n\n')

In [ ]:
!export SQUAD_DIR=/path/to/SQUAD

python run_squad.py \
  --model_type bert \
  --model_name_or_path bert-base-cased \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v1.1.json \
  --predict_file $SQUAD_DIR/dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /tmp/debug_squad/